In [3]:
import yfinance as yf
import numpy as np
import pandas as pd
from datetime import datetime, timedelta
import time
import matplotlib.pyplot as plt
import itertools

In [4]:
stockin=input('Enter Stock to be evaluated')
ticker=yf.Ticker(stockin)
today = datetime.now()
prev_day = today - timedelta(days=1)
prev_day_str = prev_day.strftime('%Y-%m-%d')

hist = ticker.history(start=prev_day_str, end=today.strftime('%Y-%m-%d'), interval='1d')

if not hist.empty:
    prev_open = hist['Open'].iloc[0]
    prev_close = hist['Close'].iloc[0]
    print(f"Previous day open: {prev_open}")
    print(f"Previous day close: {prev_close}")
else:
    print("No data for previous day.")

Previous day open: 211.77999877929688
Previous day close: 213.24000549316406


In [5]:
prev_day_mean=(prev_close+prev_open)/2
prev_day_mean

212.51000213623047

In [6]:
df = yf.download(tickers=stockin, period='1d', interval='1m')

df.dropna(inplace=True)

df['RollingMean_20'] = df['Close'].shift(1).rolling(window=20).mean()
df['RollingStd_20'] = df['Close'].shift(1).rolling(window=20).std()

current_price = df['Close'].iloc[-1]
rolling_mean_20min = df['RollingMean_20'].iloc[-1]
rolling_std_20min = df['RollingStd_20'].iloc[-1]

print("Current Price:          ", current_price)
print("20-min Rolling Mean:    ", rolling_mean_20min)
print("20-min Rolling Std Dev: ", rolling_std_20min)

C:\Users\sauri\AppData\Local\Temp\ipykernel_20076\1151132504.py:1: FutureWarning: YF.download() has changed argument auto_adjust default to True
  df = yf.download(tickers=stockin, period='1d', interval='1m')
[*********************100%***********************]  1 of 1 completed

Current Price:           Ticker
AMZN    213.240005
Name: 2025-06-12 19:59:00+00:00, dtype: float64
20-min Rolling Mean:     212.74804306030273
20-min Rolling Std Dev:  0.1311224868725344


In [7]:
z_score=(current_price-rolling_mean_20min) / rolling_std_20min
max_vol_20min = df['Volume'].shift(1).tail(20).max()
current_volume = df['Volume'].iloc[-1]

print("Current Tick Volume:", current_volume)
print("Max Volume in Last 20 Min (excluding current tick):", max_vol_20min)
print("z-score",z_score)

Current Tick Volume: Ticker
AMZN    515982
Name: 2025-06-12 19:59:00+00:00, dtype: int64
Max Volume in Last 20 Min (excluding current tick): Ticker
AMZN    278957.0
dtype: float64
z-score Ticker
AMZN    3.75193
Name: 2025-06-12 19:59:00+00:00, dtype: float64


In [8]:
# Initial capital and trading parameters
amount = 10000
initial_amount = amount
position = 0
shares_held = 0
buy_percentage = 0.95
sell_percentage = 0.30
target_profit = initial_amount * 0.20 

In [9]:
def buy_sig(z_score, max_vol_20min, current_volume):
    z_score = float(z_score.iloc[-1]) if hasattr(z_score, 'iloc') else float(z_score)
    max_vol_20min = float(max_vol_20min.iloc[-1]) if hasattr(max_vol_20min, 'iloc') else float(max_vol_20min)
    current_volume = float(current_volume.iloc[-1]) if hasattr(current_volume, 'iloc') else float(current_volume)
    
    signal = 0
    signal_type = "HOLD"
    
    volume_margin = abs(current_volume - max_vol_20min) / max_vol_20min <= 0.15
    
    if z_score <= -0.3:
        signal = 1
        signal_type = "BUY"
    elif z_score > 1.3:
        signal = -1  
        signal_type = "SELL"

    elif volume_margin and (z_score <= 1.3) and (z_score>-0.3):
            signal = -1
            signal_type = "SELL"
    elif volume_margin and (z_score>-0.3):
         signal=1
         signal_type="BUY"
    
    print(f"\nSignal Analysis:")
    print(f"Z-Score: {z_score:.4f}")
    print(f"Signal: {signal} ({signal_type})")
    print(f"Current Volume: {current_volume}")
    print(f"Max Volume (20min): {max_vol_20min}")
    print(f"Volume within 15% margin: {volume_margin}")
    
    return signal

In [10]:
# Get trading signal
signal = buy_sig(z_score, max_vol_20min, current_volume)

# Convert Series to float if needed
current_price = float(current_price.iloc[-1]) if hasattr(current_price, 'iloc') else float(current_price)

# Execute trades based on signal
if signal == 1 and position == 0:  # Buy signal
    investment = amount * buy_percentage
    shares_to_buy = int(investment // current_price)  # Convert to int
    if shares_to_buy > 0:
        position = current_price * shares_to_buy
        shares_held = shares_to_buy
        amount -= position
        print(f"Buying {shares_to_buy} shares at {current_price}")
        print(f"Position: {position}, Remaining cash: {amount}")

elif signal == -1 and position > 0:  # Sell signal
    # If z-score > 1.8 and volume condition met, sell all shares
    if z_score > 1.8 and abs(current_volume - max_vol_20min) / max_vol_20min <= 0.20:
        shares_to_sell = shares_held  # Sell all shares
    else:
        shares_to_sell = int(shares_held * sell_percentage)  # Sell normal percentage
        
    if shares_to_sell > 0:
        sale_value = current_price * shares_to_sell
        amount += sale_value
        shares_held -= shares_to_sell
        position = current_price * shares_held
        print(f"Selling {shares_to_sell} shares at {current_price}")
        print(f"Position: {position}, Available cash: {amount}")

# Check termination conditions
total_value = amount + position
profit = total_value - initial_amount

if amount <= 0:
    print("Trading stopped: No funds available")
elif profit >= target_profit:
    print(f"Trading stopped: Profit target reached. Total profit: {profit}")

print(f"Current total value: {total_value}")
print(f"Current profit/loss: {profit}")


Signal Analysis:
Z-Score: 3.7519
Signal: -1 (SELL)
Current Volume: 515982.0
Max Volume (20min): 278957.0
Volume within 15% margin: False
Current total value: 10000
Current profit/loss: 0


### MAIN CODE RUN

In [ ]:
def is_market_open():
    # Indian market hours (9:15 AM to 3:30 PM IST)
    now = datetime.now()
    market_start = now.replace(hour=9, minute=15, second=0)
    market_end = now.replace(hour=15, minute=30, second=0)
    return market_start <= now <= market_end and now.weekday() < 5

def wait_for_next_minute():
    """Wait until the start of the next minute"""
    now = datetime.now()
    # Wait until the next minute
    wait_seconds = 60 - now.second
    if wait_seconds > 0:
        print(f"Synchronizing with market ticks... waiting {wait_seconds} seconds")
        time.sleep(wait_seconds)

def run_trading_strategy():
    global amount, position, shares_held
    trades = []
    
    # Synchronize with market ticks
    wait_for_next_minute()
    
    while True:
        try:
            if not is_market_open():
                print("Market is closed. Waiting...")
                time.sleep(60)
                continue
            
            end_time = datetime.now().replace(second=0, microsecond=0)
            start_time = end_time - timedelta(hours=1)
            
            start_timestamp = int(start_time.timestamp())
            end_timestamp = int(end_time.timestamp())
            
            try:
                df = yf.download(
                    tickers=stockin,
                    start=start_timestamp,
                    end=end_timestamp,
                    interval='1m',
                    progress=False
                )
            except Exception as e:
                print(f"Download error: {e}")
                time.sleep(60)
                continue
            
            if df.empty:
                print("No data received, waiting for next tick...")
                time.sleep(60)
                continue

            df['RollingMean_20'] = df['Close'].shift(1).rolling(window=20).mean()
            df['RollingStd_20'] = df['Close'].shift(1).rolling(window=20).std()
            
            current_price = float(df['Close'].iloc[-1])
            rolling_mean_20min = df['RollingMean_20'].iloc[-1]
            rolling_std_20min = df['RollingStd_20'].iloc[-1]
            z_score = (current_price - rolling_mean_20min) / rolling_std_20min
            
            if len(df) < 2:
                print("Not enough data for previous tick's volume, waiting...")
                time.sleep(60)
                continue
            
            prev_volume = float(df['Volume'].iloc[-2])
            recent_volumes = df['Volume'].iloc[-21:-1]  
            non_zero_volumes = recent_volumes[recent_volumes > 0]
            max_vol_20min = non_zero_volumes.max() if not non_zero_volumes.empty else 0
            
            if prev_volume == 0:
                print("Warning: Previous tick's volume is 0, using last non-zero volume")
                prev_volume = non_zero_volumes.iloc[-1] if not non_zero_volumes.empty else 0
            
            signal = buy_sig(z_score, max_vol_20min, prev_volume)
            
            if signal == 1 and position == 0:  
                investment = amount * buy_percentage
                shares_to_buy = int(investment // current_price)
                if shares_to_buy > 0:
                    position = current_price * shares_to_buy
                    shares_held = shares_to_buy
                    amount -= position
                    trades.append({
                        'time': datetime.now(),
                        'type': 'BUY',
                        'shares': shares_to_buy,
                        'price': current_price,
                        'value': position,
                        'remaining_cash': amount
                    })
                    print(f"\nTRADE EXECUTED - BUY:")
                    print(f"Shares: {shares_to_buy}")
                    print(f"Price: {current_price:.2f}")
                    print(f"Position: {position:.2f}")
                    print(f"Remaining Cash: {amount:.2f}")

            elif signal == -1 and position > 0:  
                shares_to_sell = max(1, int(np.ceil(shares_held * sell_percentage))) if shares_held > 0 else 0
                if shares_to_sell > 0:
                    sale_value = current_price * shares_to_sell
                    amount += sale_value
                    shares_held -= shares_to_sell
                    position = current_price * shares_held
                    trades.append({
                        'time': datetime.now(),
                        'type': 'SELL',
                        'shares': shares_to_sell,
                        'price': current_price,
                        'value': sale_value,
                        'remaining_cash': amount
                    })
                    print(f"\nTRADE EXECUTED - SELL:")
                    print(f"Shares: {shares_to_sell}")
                    print(f"Price: {current_price:.2f}")
                    print(f"Position: {position:.2f}")
                    print(f"Available Cash: {amount:.2f}")

            total_value = amount + position
            profit = total_value - initial_amount
            
            print(f"\nPORTFOLIO STATUS:")
            print(f"Time: {datetime.now()}")
            print(f"Current Price: {current_price:.2f}")
            print(f"Shares Held: {shares_held}")
            print(f"Position Value: {position:.2f}")
            print(f"Cash: {amount:.2f}")
            print(f"Total Value: {total_value:.2f}")
            print(f"Profit/Loss: {profit:.2f}")
            print("-" * 50)
            
            if amount <= 0:
                print("\nTrading stopped: No funds available")
                break
            elif profit >= target_profit:
                print(f"\nTrading stopped: Profit target reached. Total profit: {profit:.2f}")
                break
            
            if trades:
                print("\nTRADE HISTORY:")
                for trade in trades[-3:]:  # Show last 3 trades
                    print(f"{trade['time']} - {trade['type']}: {trade['shares']} shares @ {trade['price']:.2f}")
            
            wait_for_next_minute()
            
        except Exception as e:
            print(f"Error occurred: {e}")
            time.sleep(60)

run_trading_strategy()

Synchronizing with market ticks... waiting 52 seconds


C:\Users\sauri\AppData\Local\Temp\ipykernel_12564\3517758229.py:38: FutureWarning: YF.download() has changed argument auto_adjust default to True
  df = yf.download(
C:\Users\sauri\AppData\Local\Temp\ipykernel_12564\3517758229.py:58: FutureWarning: Calling float on a single element Series is deprecated and will raise a TypeError in the future. Use float(ser.iloc[0]) instead
  current_price = float(df['Close'].iloc[-1])
C:\Users\sauri\AppData\Local\Temp\ipykernel_12564\3517758229.py:68: FutureWarning: Calling float on a single element Series is deprecated and will raise a TypeError in the future. Use float(ser.iloc[0]) instead
  prev_volume = float(df['Volume'].iloc[-2])
C:\Users\sauri\AppData\Local\Temp\ipykernel_12564\3517758229.py:58: FutureWarning: Calling float on a single element Series is deprecated and will raise a TypeError in the future. Use float(ser.iloc[0]) instead
  current_price = float(df['Close'].iloc[-1])
C:\Users\sauri\AppData\Local\Temp\ipykernel_12564\3517758229.py:


Signal Analysis:
Z-Score: -1.3133
Signal: 1 (BUY)
Current Volume: 2031.0
Max Volume (20min): 8959.0
Volume within 15% margin: False

PORTFOLIO STATUS:
Time: 2025-06-13 14:16:00.461403
Current Price: 1653.50
Shares Held: 5
Position Value: 8268.00
Cash: 1732.00
Total Value: 10000.00
Profit/Loss: 0.00
--------------------------------------------------
Synchronizing with market ticks... waiting 60 seconds


C:\Users\sauri\AppData\Local\Temp\ipykernel_12564\3517758229.py:38: FutureWarning: YF.download() has changed argument auto_adjust default to True
  df = yf.download(
C:\Users\sauri\AppData\Local\Temp\ipykernel_12564\3517758229.py:58: FutureWarning: Calling float on a single element Series is deprecated and will raise a TypeError in the future. Use float(ser.iloc[0]) instead
  current_price = float(df['Close'].iloc[-1])
C:\Users\sauri\AppData\Local\Temp\ipykernel_12564\3517758229.py:68: FutureWarning: Calling float on a single element Series is deprecated and will raise a TypeError in the future. Use float(ser.iloc[0]) instead
  prev_volume = float(df['Volume'].iloc[-2])
C:\Users\sauri\AppData\Local\Temp\ipykernel_12564\3517758229.py:58: FutureWarning: Calling float on a single element Series is deprecated and will raise a TypeError in the future. Use float(ser.iloc[0]) instead
  current_price = float(df['Close'].iloc[-1])
C:\Users\sauri\AppData\Local\Temp\ipykernel_12564\3517758229.py:


Signal Analysis:
Z-Score: -1.0909
Signal: 1 (BUY)
Current Volume: 1622.0
Max Volume (20min): 8959.0
Volume within 15% margin: False

PORTFOLIO STATUS:
Time: 2025-06-13 14:17:00.742044
Current Price: 1653.70
Shares Held: 5
Position Value: 8268.00
Cash: 1732.00
Total Value: 10000.00
Profit/Loss: 0.00
--------------------------------------------------
Synchronizing with market ticks... waiting 60 seconds


...


Signal Analysis:
Z-Score: 2.4792
Signal: -1 (SELL)
Current Volume: 25208.0
Max Volume (20min): 47724.0
Volume within 15% margin: False

PORTFOLIO STATUS:
Time: 2025-06-13 15:28:00.332196
Current Price: 1664.00
Shares Held: 0
Position Value: 0.00
Cash: 10014.60
Total Value: 10014.60
Profit/Loss: 14.60
--------------------------------------------------

TRADE HISTORY:
2025-06-13 14:43:01.107578 - SELL: 1 shares @ 1655.40
2025-06-13 14:44:00.443675 - SELL: 1 shares @ 1655.70
2025-06-13 15:03:01.155009 - SELL: 1 shares @ 1654.50
Synchronizing with market ticks... waiting 60 seconds


C:\Users\sauri\AppData\Local\Temp\ipykernel_12564\3517758229.py:38: FutureWarning: YF.download() has changed argument auto_adjust default to True
  df = yf.download(
C:\Users\sauri\AppData\Local\Temp\ipykernel_12564\3517758229.py:58: FutureWarning: Calling float on a single element Series is deprecated and will raise a TypeError in the future. Use float(ser.iloc[0]) instead
  current_price = float(df['Close'].iloc[-1])
C:\Users\sauri\AppData\Local\Temp\ipykernel_12564\3517758229.py:68: FutureWarning: Calling float on a single element Series is deprecated and will raise a TypeError in the future. Use float(ser.iloc[0]) instead
  prev_volume = float(df['Volume'].iloc[-2])
C:\Users\sauri\AppData\Local\Temp\ipykernel_12564\3517758229.py:58: FutureWarning: Calling float on a single element Series is deprecated and will raise a TypeError in the future. Use float(ser.iloc[0]) instead
  current_price = float(df['Close'].iloc[-1])
C:\Users\sauri\AppData\Local\Temp\ipykernel_12564\3517758229.py:


Signal Analysis:
Z-Score: 0.7601
Signal: 0 (HOLD)
Current Volume: 37433.0
Max Volume (20min): 47724.0
Volume within 15% margin: False

PORTFOLIO STATUS:
Time: 2025-06-13 15:29:00.608126
Current Price: 1661.90
Shares Held: 0
Position Value: 0.00
Cash: 10014.60
Total Value: 10014.60
Profit/Loss: 14.60
--------------------------------------------------

TRADE HISTORY:
2025-06-13 14:43:01.107578 - SELL: 1 shares @ 1655.40
2025-06-13 14:44:00.443675 - SELL: 1 shares @ 1655.70
2025-06-13 15:03:01.155009 - SELL: 1 shares @ 1654.50
Synchronizing with market ticks... waiting 60 seconds


C:\Users\sauri\AppData\Local\Temp\ipykernel_12564\3517758229.py:38: FutureWarning: YF.download() has changed argument auto_adjust default to True
  df = yf.download(
C:\Users\sauri\AppData\Local\Temp\ipykernel_12564\3517758229.py:58: FutureWarning: Calling float on a single element Series is deprecated and will raise a TypeError in the future. Use float(ser.iloc[0]) instead
  current_price = float(df['Close'].iloc[-1])
C:\Users\sauri\AppData\Local\Temp\ipykernel_12564\3517758229.py:68: FutureWarning: Calling float on a single element Series is deprecated and will raise a TypeError in the future. Use float(ser.iloc[0]) instead
  prev_volume = float(df['Volume'].iloc[-2])



Signal Analysis:
Z-Score: 0.7601
Signal: 0 (HOLD)
Current Volume: 37433.0
Max Volume (20min): 47724.0
Volume within 15% margin: False

PORTFOLIO STATUS:
Time: 2025-06-13 15:30:00.770363
Current Price: 1661.90
Shares Held: 0
Position Value: 0.00
Cash: 10014.60
Total Value: 10014.60
Profit/Loss: 14.60
--------------------------------------------------

TRADE HISTORY:
2025-06-13 14:43:01.107578 - SELL: 1 shares @ 1655.40
2025-06-13 14:44:00.443675 - SELL: 1 shares @ 1655.70
2025-06-13 15:03:01.155009 - SELL: 1 shares @ 1654.50
Synchronizing with market ticks... waiting 60 seconds
Market is closed. Waiting...
Market is closed. Waiting...
Market is closed. Waiting...
Market is closed. Waiting...
Market is closed. Waiting...
Market is closed. Waiting...
Market is closed. Waiting...
Market is closed. Waiting...
Market is closed. Waiting...
Market is closed. Waiting...


### BACKTESTING


In [11]:
def backtest_strategy(symbol, start_date, end_date, initial_capital=10000):
    # Download historical data
    df = yf.download(symbol, start=start_date, end=end_date, interval='1m')
    
    if df.empty:
        print('No data available for the specified period')
        return
    
    # Initialize variables
    amount = initial_capital
    initial_amount = amount
    position = 0
    shares_held = 0
    trades = []
    
    df['RollingMean_20'] = df['Close'].shift(1).rolling(window=20).mean()
    df['RollingStd_20'] = df['Close'].shift(1).rolling(window=20).std()
    
    df = df.dropna()
    
    for i in range(len(df)):
        current_price = float(df['Close'].iloc[i])
        rolling_mean_20min = df['RollingMean_20'].iloc[i]
        rolling_std_20min = df['RollingStd_20'].iloc[i]
        
        z_score = (current_price - rolling_mean_20min) / rolling_std_20min
        
        if i >= 20:
            recent_volumes = df['Volume'].iloc[i-20:i]
            max_vol_20min = recent_volumes.max()
        else:
            max_vol_20min = df['Volume'].iloc[:i].max()
        
        current_volume = df['Volume'].iloc[i]
        
        signal = buy_sig(z_score, max_vol_20min, current_volume)
        
        if signal == 1 and position == 0:  
            investment = amount * buy_percentage
            shares_to_buy = int(investment // current_price)
            if shares_to_buy > 0:
                position = current_price * shares_to_buy
                shares_held = shares_to_buy
                amount -= position
                trades.append({
                    'time': df.index[i],
                    'type': 'BUY',
                    'shares': shares_to_buy,
                    'price': current_price,
                    'value': position,
                    'remaining_cash': amount
                })

        elif signal == -1 and position > 0:  
            shares_to_sell = max(1, int(np.ceil(shares_held * sell_percentage))) if shares_held > 0 else 0
            if shares_to_sell > 0:
                sale_value = current_price * shares_to_sell
                amount += sale_value
                shares_held -= shares_to_sell
                position = current_price * shares_held
                trades.append({
                    'time': df.index[i],
                    'type': 'SELL',
                    'shares': shares_to_sell,
                    'price': current_price,
                    'value': sale_value,
                    'remaining_cash': amount
                })
        
        if shares_held > 0:
            position = current_price * shares_held
    
    total_value = amount + position
    total_return = ((total_value - initial_capital) / initial_capital) * 100
    
    print(f'\nBacktest Results ({start_date} to {end_date}):')
    print(f'Initial Capital: ${initial_amount:,.2f}')
    print(f'Final Capital: ${total_value:,.2f}')
    print(f'Total Return: {total_return:.2f}%')
    print(f'Number of Trades: {len(trades)}')
    
    # Print trade history
    print('\nTrade History:')
    for trade in trades:
        print(f"{trade['time']} - {trade['type']}: {trade['shares']} shares @ ${trade['price']:.2f}")
    
    return df, trades

In [12]:
from datetime import datetime, timedelta

end_date = datetime.now()
start_date = end_date - timedelta(days=4)  

df, trades = backtest_strategy(stockin, start_date, end_date, initial_capital=10000)

C:\Users\sauri\AppData\Local\Temp\ipykernel_20076\1520823160.py:3: FutureWarning: YF.download() has changed argument auto_adjust default to True
  df = yf.download(symbol, start=start_date, end=end_date, interval='1m')
[*********************100%***********************]  1 of 1 completed
C:\Users\sauri\AppData\Local\Temp\ipykernel_20076\1520823160.py:22: FutureWarning: Calling float on a single element Series is deprecated and will raise a TypeError in the future. Use float(ser.iloc[0]) instead
  current_price = float(df['Close'].iloc[i])

C:\Users\sauri\AppData\Local\Temp\ipykernel_20076\1520823160.py:22: FutureWarning: Calling float on a single element Series is deprecated and will raise a TypeError in the future. Use float(ser.iloc[0]) instead
  current_price = float(df['Close'].iloc[i])



Signal Analysis:
Z-Score: -1.3019
Signal: 1 (BUY)
Current Volume: 234525.0
Max Volume (20min): nan
Volume within 15% margin: False

Signal Analysis:
Z-Score: -0.3090
Signal: 1 (BUY)
Current Volume: 313033.0
Max Volume (20min): 234525.0
Volume within 15% margin: False

Signal Analysis:
Z-Score: -2.5692
Signal: 1 (BUY)
Current Volume: 767251.0
Max Volume (20min): 313033.0
Volume within 15% margin: False

Signal Analysis:
Z-Score: -3.1304
Signal: 1 (BUY)
Current Volume: 1482362.0
Max Volume (20min): 767251.0
Volume within 15% margin: False

Signal Analysis:
Z-Score: -3.9716
Signal: 1 (BUY)
Current Volume: 256320.0
Max Volume (20min): 1482362.0
Volume within 15% margin: False

Signal Analysis:
Z-Score: -1.8881
Signal: 1 (BUY)
Current Volume: 163813.0
Max Volume (20min): 1482362.0
Volume within 15% margin: False

Signal Analysis:
Z-Score: -0.9326
Signal: 1 (BUY)
Current Volume: 153943.0
Max Volume (20min): 1482362.0
Volume within 15% margin: False

Signal Analysis:
Z-Score: -1.0627
Signal:

### Z-Score Optimization

In [ ]:
def optimize_z_scores(symbol, start_date, end_date, initial_capital=10000):
    buy_thresholds = np.arange(-1.5, -0.2, 0.1)   
    sell_thresholds = np.arange(1.1, 2.0, 0.1)   
    
    best_return = -float('inf')
    best_params = None
    results = []
    
    try:
        df = yf.download(symbol, start=start_date, end=end_date, interval='1m')
        if df.empty:
            print(f'No data available for {symbol} in the specified period')
            return None, pd.DataFrame()
    except Exception as e:
        print(f'Error downloading data: {e}')
        return None, pd.DataFrame()
    
    def modified_buy_sig(z_score, max_vol_20min, current_volume, buy_threshold, sell_threshold):
        z_score = float(z_score.iloc[-1]) if hasattr(z_score, 'iloc') else float(z_score)
        max_vol_20min = float(max_vol_20min.iloc[-1]) if hasattr(max_vol_20min, 'iloc') else float(max_vol_20min)
        current_volume = float(current_volume.iloc[-1]) if hasattr(current_volume, 'iloc') else float(current_volume)
        
        signal = 0
        
        if z_score <= buy_threshold:
            signal = 1
        elif z_score >= sell_threshold:
            signal = -1
        elif (z_score > 0) and (z_score <= sell_threshold):
            if max_vol_20min > 0 and abs(current_volume - max_vol_20min) / max_vol_20min <= 0.15:
                signal = 1
        
        return signal
    
    df['RollingMean_20'] = df['Close'].shift(1).rolling(window=20).mean()
    df['RollingStd_20'] = df['Close'].shift(1).rolling(window=20).std()
    df = df.dropna()
    
    print(f'Testing {len(buy_thresholds) * len(sell_thresholds)} combinations...')
    
    for buy_thresh in buy_thresholds:
        for sell_thresh in sell_thresholds:
            amount = initial_capital
            position = 0
            shares_held = 0
            trades = []
            
            for i in range(len(df)):
                current_price = float(df['Close'].iloc[i])
                rolling_mean = df['RollingMean_20'].iloc[i]
                rolling_std = df['RollingStd_20'].iloc[i]
                z_score = (current_price - rolling_mean) / rolling_std
                
                if i >= 20:
                    recent_volumes = df['Volume'].iloc[i-20:i]
                    max_vol_20min = float(recent_volumes.max())
                else:
                    max_vol_20min = float(df['Volume'].iloc[:i].max()) if i > 0 else 0.0
                
                current_volume = float(df['Volume'].iloc[i])
                
                signal = modified_buy_sig(z_score, max_vol_20min, current_volume, buy_thresh, sell_thresh)
                
                if signal == 1 and position == 0:
                    investment = amount * buy_percentage
                    shares_to_buy = int(investment // current_price)
                    if shares_to_buy > 0:
                        position = current_price * shares_to_buy
                        shares_held = shares_to_buy
                        amount -= position
                        trades.append({
                            'time': df.index[i],
                            'type': 'BUY',
                            'shares': shares_to_buy,
                            'price': current_price
                        })
                
                elif signal == -1 and position > 0:
                    shares_to_sell = int(shares_held * sell_percentage)
                    if shares_to_sell > 0:
                        sale_value = current_price * shares_to_sell
                        amount += sale_value
                        shares_held -= shares_to_sell
                        position = current_price * shares_held
                        trades.append({
                            'time': df.index[i],
                            'type': 'SELL',
                            'shares': shares_to_sell,
                            'price': current_price
                        })
            
            final_position_value = shares_held * float(df['Close'].iloc[-1])
            total_value = amount + final_position_value
            total_return = ((total_value - initial_capital) / initial_capital) * 100
            
            results.append({
                'buy_threshold': buy_thresh,
                'sell_threshold': sell_thresh,
                'total_return': total_return,
                'n_trades': len(trades)
            })
            
            if total_return > best_return:
                best_return = total_return
                best_params = {
                    'buy_threshold': buy_thresh,
                    'sell_threshold': sell_thresh,
                    'total_return': total_return,
                    'n_trades': len(trades)
                }
    
    results_df = pd.DataFrame(results)
    return best_params, results_df

In [ ]:
end_date = datetime.now()
start_date = end_date - timedelta(days=5)  

print('Running optimization for the last 5 days...')
print('Note: Using 1-minute data from the last 5 trading days due to Yahoo Finance API limitations')
best_params, results_df = optimize_z_scores(stockin, start_date, end_date, initial_capital=10000)

print('\nBest Parameters Found:')
print(f"Buy Threshold (z-score): {best_params['buy_threshold']:.2f}")
print(f"Sell Threshold (z-score): {best_params['sell_threshold']:.2f}")
print(f"Total Return: {best_params['total_return']:.2f}%")
print(f"Number of Trades: {best_params['n_trades']}")

print('\nTop 5 Combinations:')
print(results_df.sort_values('total_return', ascending=False).head())

import seaborn as sns
import matplotlib.pyplot as plt

heatmap_data = results_df.pivot(index='buy_threshold', columns='sell_threshold', values='total_return')

plt.figure(figsize=(12, 8))
sns.heatmap(heatmap_data, annot=True, cmap='RdYlGn', center=0)
plt.title('Returns Heatmap: Buy vs Sell Thresholds')
plt.xlabel('Sell Threshold')
plt.ylabel('Buy Threshold')
plt.show()

### BUY/SELL OPTIMIZATION

In [ ]:
def optimize_percentages(symbol, start_date, end_date, initial_capital=10000):
    # Define the ranges to test
    buy_percentages = np.arange(0.1, 1.0, 0.05)   
    sell_percentages = np.arange(0.1, 1.0, 0.05)  
    
    best_return = -float('inf')
    best_params = None
    results = []
    
    try:
        df = yf.download(symbol, start=start_date, end=end_date, interval='1m')
        if df.empty:
            print(f'No data available for {symbol} in the specified period')
            return None, pd.DataFrame()
    except Exception as e:
        print(f'Error downloading data: {e}')
        return None, pd.DataFrame()
    
    df['RollingMean_20'] = df['Close'].shift(1).rolling(window=20).mean()
    df['RollingStd_20'] = df['Close'].shift(1).rolling(window=20).std()
    df = df.dropna()
    
    print(f'Testing {len(buy_percentages) * len(sell_percentages)} combinations...')
    
    for buy_pct in buy_percentages:
        for sell_pct in sell_percentages:
            amount = initial_capital
            position = 0
            shares_held = 0
            trades = []
            
            for i in range(len(df)):
                current_price = float(df['Close'].iloc[i])
                rolling_mean = df['RollingMean_20'].iloc[i]
                rolling_std = df['RollingStd_20'].iloc[i]
                z_score = (current_price - rolling_mean) / rolling_std
                
                if i >= 20:
                    recent_volumes = df['Volume'].iloc[i-20:i]
                    max_vol_20min = float(recent_volumes.max())
                else:
                    max_vol_20min = float(df['Volume'].iloc[:i].max()) if i > 0 else 0.0
                
                current_volume = float(df['Volume'].iloc[i])
                
                signal = buy_sig(z_score, max_vol_20min, current_volume)
                
                if signal == 1 and position == 0:
                    investment = amount * buy_pct
                    shares_to_buy = int(investment // current_price)
                    if shares_to_buy > 0:
                        position = current_price * shares_to_buy
                        shares_held = shares_to_buy
                        amount -= position
                        trades.append({
                            'time': df.index[i],
                            'type': 'BUY',
                            'shares': shares_to_buy,
                            'price': current_price
                        })
                
                elif signal == -1 and position > 0:
                    shares_to_sell = int(shares_held * sell_pct)
                    if shares_to_sell > 0:
                        sale_value = current_price * shares_to_sell
                        amount += sale_value
                        shares_held -= shares_to_sell
                        position = current_price * shares_held
                        trades.append({
                            'time': df.index[i],
                            'type': 'SELL',
                            'shares': shares_to_sell,
                            'price': current_price
                        })
            
            final_position_value = shares_held * float(df['Close'].iloc[-1])
            total_value = amount + final_position_value
            total_return = ((total_value - initial_capital) / initial_capital) * 100
            
            results.append({
                'buy_percentage': buy_pct * 100,  
                'sell_percentage': sell_pct * 100,
                'total_return': total_return,
                'n_trades': len(trades)
            })
            
            if total_return > best_return:
                best_return = total_return
                best_params = {
                    'buy_percentage': buy_pct * 100,
                    'sell_percentage': sell_pct * 100,
                    'total_return': total_return,
                    'n_trades': len(trades)
                }
    
    results_df = pd.DataFrame(results)
    return best_params, results_df

In [ ]:
print('\nOptimizing trading percentages...')
print('Note: Using 1-minute data from the last 5 trading days')
best_pct_params, pct_results_df = optimize_percentages(stockin, start_date, end_date, initial_capital=10000)

print('\nBest Percentage Parameters Found:')
print(f"Buy Percentage: {best_pct_params['buy_percentage']:.1f}%")
print(f"Sell Percentage: {best_pct_params['sell_percentage']:.1f}%")
print(f"Total Return: {best_pct_params['total_return']:.2f}%")
print(f"Number of Trades: {best_pct_params['n_trades']}")

print('\nTop 5 Percentage Combinations:')
print(pct_results_df.sort_values('total_return', ascending=False).head())

plt.figure(figsize=(12, 8))
heatmap_data_pct = pct_results_df.pivot(
    index='buy_percentage',
    columns='sell_percentage',
    values='total_return'
)
sns.heatmap(heatmap_data_pct, annot=True, cmap='RdYlGn', center=0, fmt='.1f')
plt.title('Returns Heatmap: Buy vs Sell Percentages')
plt.xlabel('Sell Percentage')
plt.ylabel('Buy Percentage')
plt.show()

### STOCK, Z-Score, BUY/SELL OPTIMIZATION

## Single Thread

In [ ]:
stocks = ['AAPL', 'MSFT', 'GOOGL', 'AMZN', 'TSLA', 'META', 'RELIANCE.NS', 'TCS.NS', 'INFY.NS', 'HDFCBANK.NS']

buy_z_scores = np.arange(-0.3, -0.8, -0.1)
sell_z_scores = np.arange(1.1, 1.4, 0.1)
buy_percentages = np.arange(0.20, 1.00, 0.05)
sell_percentages = np.arange(0.20, 1.00, 0.05)

results = []

global_start_date = datetime.now() - timedelta(days=3)
global_end_date = datetime.now()

for stock in stocks:
    print(f"\nProcessing {stock}...")
    try:
        df = yf.download(stock, start=global_start_date, end=global_end_date, interval='1m', progress=False)
        if df.empty:
            print(f"No data for {stock}")
            continue
        df['RollingMean_20'] = df['Close'].shift(1).rolling(window=20).mean()
        df['RollingStd_20'] = df['Close'].shift(1).rolling(window=20).std()
        df = df.dropna()
    except Exception as e:
        print(f"Error downloading {stock}: {e}")
        continue

    for buy_z, sell_z, buy_pct, sell_pct in itertools.product(buy_z_scores, sell_z_scores, buy_percentages, sell_percentages):
        amount = 10000
        position = 0
        shares_held = 0
        for i in range(len(df)):
            current_price = float(df['Close'].iloc[i])
            rolling_mean = df['RollingMean_20'].iloc[i]
            rolling_std = df['RollingStd_20'].iloc[i]
            z_score = (current_price - rolling_mean) / rolling_std
            if i >= 20:
                recent_volumes = df['Volume'].iloc[i-20:i]
                max_vol_20min = float(recent_volumes.max())
            else:
                max_vol_20min = float(df['Volume'].iloc[:i].max()) if i > 0 else 0.0
            current_volume = float(df['Volume'].iloc[i])
            signal = 0
            if z_score <= buy_z:
                signal = 1
            elif z_score >= sell_z:
                signal = -1
            elif (z_score > 0) and (z_score <= sell_z):
                if max_vol_20min > 0 and abs(current_volume - max_vol_20min) / max_vol_20min <= 0.15:
                    signal = 1
            if signal == 1 and position == 0:
                investment = amount * buy_pct
                shares_to_buy = int(investment // current_price)
                if shares_to_buy > 0:
                    position = current_price * shares_to_buy
                    shares_held = shares_to_buy
                    amount -= position
            elif signal == -1 and position > 0:
                shares_to_sell = int(shares_held * sell_pct)
                if shares_to_sell > 0:
                    sale_value = current_price * shares_to_sell
                    amount += sale_value
                    shares_held -= shares_to_sell
                    position = current_price * shares_held
        final_position_value = shares_held * float(df['Close'].iloc[-1])
        total_value = amount + final_position_value
        total_return = ((total_value - 10000) / 10000) * 100
        results.append({
            'stock': stock,
            'buy_z': buy_z,
            'sell_z': sell_z,
            'buy_pct': buy_pct,
            'sell_pct': sell_pct,
            'total_return': total_return
        })

results_df = pd.DataFrame(results)

top20 = results_df.sort_values('total_return', ascending=False).head(20)
print("\nTop 20 Best Performing Combinations (across all stocks):")
print(top20)

for stock in stocks:
    stock_df = results_df[results_df['stock'] == stock]
    if not stock_df.empty:
        pivot = stock_df.pivot_table(index='buy_z', columns='sell_z', values='total_return', aggfunc='max')
        plt.figure(figsize=(10, 6))
        sns.heatmap(pivot, annot=False, cmap='RdYlGn', center=0)
        plt.title(f'Returns Heatmap for {stock}\n(Buy Z vs Sell Z, best buy/sell % for each)')
        plt.xlabel('Sell Z-Score')
        plt.ylabel('Buy Z-Score')
        plt.show()

best_stock = top20.iloc[0]['stock']
best_stock_df = results_df[results_df['stock'] == best_stock]
pivot_pct = best_stock_df.pivot_table(index='buy_pct', columns='sell_pct', values='total_return', aggfunc='max')
plt.figure(figsize=(10, 6))
sns.heatmap(pivot_pct, annot=False, cmap='RdYlGn', center=0)
plt.title(f'Returns Heatmap for {best_stock}\n(Buy % vs Sell %, best z-scores for each)')
plt.xlabel('Sell Percentage')
plt.ylabel('Buy Percentage')
plt.show()

## Parallel Threading


In [ ]:
from concurrent.futures import ProcessPoolExecutor, as_completed
from concurrent.futures import ThreadPoolExecutor, as_completed


def simulate_strategy(stock, df, buy_z, sell_z, buy_pct, sell_pct):
    amount = 10000
    position = 0
    shares_held = 0
    for i in range(len(df)):
        current_price = float(df['Close'].iloc[i])
        rolling_mean = df['RollingMean_20'].iloc[i]
        rolling_std = df['RollingStd_20'].iloc[i]
        z_score = (current_price - rolling_mean) / rolling_std

        if i >= 20:
            recent_volumes = df['Volume'].iloc[i-20:i]
            max_vol_20min = float(recent_volumes.max())
        else:
            max_vol_20min = float(df['Volume'].iloc[:i].max()) if i > 0 else 0.0

        current_volume = float(df['Volume'].iloc[i])
        signal = 0
        if z_score <= buy_z:
            signal = 1
        elif z_score >= sell_z:
            signal = -1
        elif (z_score > 0) and (z_score <= sell_z):
            if max_vol_20min > 0 and abs(current_volume - max_vol_20min) / max_vol_20min <= 0.15:
                signal = 1

        if signal == 1 and position == 0:
            investment = amount * buy_pct
            shares_to_buy = int(investment // current_price)
            if shares_to_buy > 0:
                position = current_price * shares_to_buy
                shares_held = shares_to_buy
                amount -= position
        elif signal == -1 and position > 0:
            shares_to_sell = int(shares_held * sell_pct)
            if shares_to_sell > 0:
                sale_value = current_price * shares_to_sell
                amount += sale_value
                shares_held -= shares_to_sell
                position = current_price * shares_held

    final_position_value = shares_held * float(df['Close'].iloc[-1])
    total_value = amount + final_position_value
    total_return = ((total_value - 10000) / 10000) * 100
    if len(results) % 100 == 0:
        pd.DataFrame(results).to_csv("partial_results.csv", index=False)
    return {
        'stock': stock,
        'buy_z': buy_z,
        'sell_z': sell_z,
        'buy_pct': buy_pct,
        'sell_pct': sell_pct,
        'total_return': total_return
    }



# ==== Parameter Setup ====
stocks = ['AAPL', 'MSFT', 'GOOGL', 'AMZN', 'TSLA', 'META', 'RELIANCE.NS', 'TCS.NS', 'INFY.NS', 'HDFCBANK.NS']
buy_z_scores = np.arange(-0.3, -0.8, -0.1)
sell_z_scores = np.arange(1.1, 1.4, 0.1)
buy_percentages = np.arange(0.20, 1.00, 0.05)
sell_percentages = np.arange(0.20, 1.00, 0.05)

global_start_date = datetime.now() - timedelta(days=3)
global_end_date = datetime.now()

# ==== Prepare Tasks ====

idx = 0
total_tasks = len(stocks) * len(buy_z_scores) * len(sell_z_scores) * len(buy_percentages) * len(sell_percentages)

tasks = []
for stock in stocks:
    print(f"\nDownloading {stock}...")
    df = yf.download(stock, start=global_start_date, end=global_end_date, interval='1m', progress=False)
    if df.empty:
        print(f"No data for {stock}")
        continue
    df['RollingMean_20'] = df['Close'].shift(1).rolling(window=20).mean()
    df['RollingStd_20'] = df['Close'].shift(1).rolling(window=20).std()
    df = df.dropna()
    for buy_z, sell_z, buy_pct, sell_pct in itertools.product(buy_z_scores, sell_z_scores, buy_percentages, sell_percentages):
        idx += 1
        if idx % 100 == 0:
            print(f"Completed {idx}/{total_tasks}")

        tasks.append((stock, df.copy(), buy_z, sell_z, buy_pct, sell_pct))

# ==== Run Parallel Simulations ====
results = []
with ThreadPoolExecutor() as executor:
    futures = [executor.submit(simulate_strategy, *task) for task in tasks]
    for future in as_completed(futures):
        try:
            results.append(future.result())
        except Exception as e:
            print(f"Error in task: {e}")

results_df = pd.DataFrame(results)

# ==== Output Top 20 ====
top20 = results_df.sort_values('total_return', ascending=False).head(20)
print("\nTop 20 Best Performing Combinations:")
print(top20)

# ==== Per Stock Heatmaps ====
for stock in stocks:
    stock_df = results_df[results_df['stock'] == stock]
    if not stock_df.empty:
        pivot = stock_df.pivot_table(index='buy_z', columns='sell_z', values='total_return', aggfunc='max')
        plt.figure(figsize=(10, 6))
        sns.heatmap(pivot, annot=False, cmap='RdYlGn', center=0)
        plt.title(f'Returns Heatmap for {stock}\n(Buy Z vs Sell Z)')
        plt.xlabel('Sell Z-Score')
        plt.ylabel('Buy Z-Score')
        plt.show()

# ==== Heatmap for Best Stock's Buy/Sell Percentages ====
if not top20.empty:
    best_stock = top20.iloc[0]['stock']
    best_stock_df = results_df[results_df['stock'] == best_stock]
    pivot_pct = best_stock_df.pivot_table(index='buy_pct', columns='sell_pct', values='total_return', aggfunc='max')
    plt.figure(figsize=(10, 6))
    sns.heatmap(pivot_pct, annot=False, cmap='RdYlGn', center=0)
    plt.title(f'Returns Heatmap for {best_stock}\n(Buy % vs Sell %)')
    plt.xlabel('Sell Percentage')
    plt.ylabel('Buy Percentage')
    plt.savefig(f"{stock}_heatmap.png")
    plt.close()

